# 0-Tokenizer

Tokenizer（分词器）在 NLP 领域扮演着基础且关键的作用，它将文本分割成单词或子词并转化为数组编号，为模型提供可处理的输入，在文本预处理、语义理解及适配不同语言和任务等方面奠定基础，是连接自然语言文本与计算机可处理数据的重要桥梁.

## 子词分词算法

常见的子词分词算法有三种：

1. 字节对编码（Byte Pair Encoding，BPE）
2. WordPiece
3. Unigram

### BPE

BPE 是一种简单的数据压缩技术，它会迭代地替换序列中最频繁出现的字节对。BPE 依赖一个预分词器，该预分词器会将训练数据分割成单词（在本项目中，我们使用按空格分词的方法作为预分词方法）.

在预分词之后，会创建一组唯一的单词，并确定它们在数据中的出现频率。接下来，BPE 会创建一个基础词表，该词表包含预分词器最初生成的数据中所有唯一单词的符号。然后，会将这对符号从词表中移除，新形成的符号将加入词表。在迭代过程中，BPE 算法会合并频繁出现的符号对.

给定词表的大小，BPE（字节对编码）算法最终会合并出现频率最高的符号对，直到收敛到该大小.

### WordPiece

WordPiece 算法与 BPE 非常相似。WordPiece 首先将词表初始化为包含训练数据中出现的每个字符，然后逐步学习给定数量的合并规则. 与 BPE 不同的是，WordPiece 并不选择最频繁出现的符号对，而是选择那个加入词表后能使训练数据出现的可能性最大化的符号对.

### Unigram

Unigram 算法将其基础词表初始化为大量的符号，然后逐步削减每个符号，以获得一个更小的词表。它会在训练数据上定义一个对数似然损失，以此来确定是否从词表中移除某个符号.

## 训练一个最简单的分词器

在本节中，我们将学习基于 transformers 库来训练你自己的分词器.

### 初始化

首先，我们应该初始化我们的分词器，并确定选择哪种方法。我们将使用字节对编码（BPE）算法.

In [1]:
from tokenizers import pre_tokenizers

alphabet = pre_tokenizers.ByteLevel.alphabet()
alphabet.sort()
print(alphabet)
print([ord(i) for i in alphabet])
print([hex(ord(i)) for i in alphabet])
# ! 一共256个字符，从33开始。因为ascii表中前32个字符是控制字符，所以从33开始。
# ascii有128个代码点，unicode的前128个代码点与ascii相同。
# ord() 获取字符的 ASCII 码
# hex() 将 ASCII 码转换为十六进制字符串

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '¡', '¢', '£', '¤', '¥', '¦', '§', '¨', '©', 'ª', '«', '¬', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¶', '·', '¸', '¹', 'º', '»', '¼', '½', '¾', '¿', 'À', 'Á', 'Â', 'Ã', 'Ä', 'Å', 'Æ', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ï', 'Ð', 'Ñ', 'Ò', 'Ó', 'Ô', 'Õ', 'Ö', '×', 'Ø', 'Ù', 'Ú', 'Û', 'Ü', 'Ý', 'Þ', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', '÷', 'ø', 'ù', 'ú', 'û', 'ü', 'ý', 'þ', 'ÿ', 'Ā', 'ā', 'Ă', 'ă', 'Ą', 'ą', 'Ć', 'ć', 'Ĉ', 'ĉ', 'Ċ', 'ċ'

In [2]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

### 定义特殊标记

数据集中存在一些我们不希望被分词的特殊标记，我们会将这些标记定义为特殊标记，并将它们传递给分词器训练器，以防止出现错误的分词情况.

In [11]:
special_tokens = ["<unk>", "<s>", "</s>"]

trainer = trainers.BpeTrainer(
        vocab_size=270, # 256 + 14
        special_tokens=special_tokens,
        show_progress=True,
        initial_alphabet=pre_tokenizers.ByteLevel.alphabet()
)

### 从文件中读取数据

在本次实验中，我们使用 JSON Lines（jsonl）格式来存储 Tokenizer 训练数据，分词器内置的训练函数要求训练数据以迭代器的形式传入，因此，我们首先获取一个数据读取的生成器.

In [12]:
import json

def read_texts_from_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            yield data['text']

data_path = './toydata/tokenizer_data.jsonl'

In [13]:
data_iter = read_texts_from_jsonl(data_path)
print(f'Row 1: {next(data_iter)}')

Row 1: <s>近年来，人工智能技术迅速发展，深刻改变了各行各业的面貌。机器学习、自然语言处理、计算机视觉等领域的突破性进展，使得智能产品和服务越来越普及。从智能家居到自动驾驶，再到智能医疗，AI的应用场景正在快速拓展。随着技术的不断进步，未来的人工智能将更加智能、更加贴近人类生活。</s>


### 开始训练!

我们使用分词器的内置函数 `tokenizer.train_from_iterator` 来训练分词器.

In [14]:
tokenizer.train_from_iterator(data_iter, trainer=trainer)

### 设置解码器

In [15]:
tokenizer.decoder = decoders.ByteLevel()

接下来，检查一下特殊标记是否得到了妥善处理。

In [16]:
assert tokenizer.token_to_id('<unk>') == 0
assert tokenizer.token_to_id('<s>') == 1
assert tokenizer.token_to_id('</s>') == 2

### 将训练好的分词器保存到磁盘

In [17]:
import os

tokenizer_dir = "./model/toy_tokenizer"
os.makedirs(tokenizer_dir, exist_ok=True)
tokenizer.save(os.path.join(tokenizer_dir, "tokenizer.json")) # At this point, you will see a file named tokenizer.json under tokenizer_dir
tokenizer.model.save(tokenizer_dir) # generate vocab.json & merges.txt

# vocab_size=256+3+11=270，给特殊标记3个，给语料中的pair 11个

['./model/toy_tokenizer/vocab.json', './model/toy_tokenizer/merges.txt']

### 手动创建一份配置文件

In [18]:
config = {
    "add_bos_token": False,
    "add_eos_token": False,
    "add_prefix_space": False,
    "added_tokens_decoder": {
        "0": {
            "content": "<unk>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
        },
        "1": {
            "content": "<s>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
        },
        "2": {
            "content": "</s>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
        }
    },
    "additional_special_tokens": [],
    "bos_token": "<s>",
    "clean_up_tokenization_spaces": False,
    "eos_token": "</s>",
    "legacy": True,
    "model_max_length": 32768,
    "pad_token": "<unk>",
    "sp_model_kwargs": {},
    "spaces_between_special_tokens": False,
    "tokenizer_class": "PreTrainedTokenizerFast",
    "unk_token": "<unk>",
    "chat_template": "{{ '<s>' + messages[0]['text'] + '</s>' }}"
}

with open(os.path.join(tokenizer_dir, "tokenizer_config.json"), "w", encoding="utf-8") as config_file:
    json.dump(config, config_file, ensure_ascii=False, indent=4)

print("Tokenizer training completed and saved.")

Tokenizer training completed and saved.


现在我们已经训练了一个简单的分词器，并将其进行保存，接下来，我们试着加载它，并使用其帮助我们对文本进行编解码.

In [19]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./model/toy_tokenizer")
msg = [{"text": "失去的东西就要学着去接受，学着放下。"}]
new_msg = tokenizer.apply_chat_template(
    msg,
    tokenize=False
)
print(f'原始文本：{msg}')
print(f'修改文本：{msg} (添加自定义聊天模板)')

/home/hey/anaconda3/envs/minimind/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


原始文本：[{'text': '失去的东西就要学着去接受，学着放下。'}]
修改文本：[{'text': '失去的东西就要学着去接受，学着放下。'}] (添加自定义聊天模板)


In [20]:
print(f'分词器词表大小：{tokenizer.vocab_size}')

分词器词表大小：270


In [21]:
model_inputs = tokenizer(new_msg)
print(f'查看分词结果：\n{model_inputs}')

查看分词结果：
{'input_ids': [1, 260, 112, 164, 239, 122, 265, 259, 253, 167, 101, 126, 164, 111, 112, 167, 102, 226, 164, 258, 102, 166, 254, 225, 164, 239, 122, 165, 239, 101, 266, 248, 264, 164, 258, 102, 166, 254, 225, 165, 245, 125, 259, 236, 269, 227, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [22]:
response = tokenizer.decode(model_inputs['input_ids'], skip_special_tokens=False)
print(f'对分词结果进行解码：{response} (保留特殊字符)' )

对分词结果进行解码：<s>失去的东西就要学着去接受，学着放下。</s> (保留特殊字符)


In [23]:
response = tokenizer.decode(model_inputs['input_ids'], skip_special_tokens=True)
print(f'对分词结果进行解码：{response} (移除特殊字符)' )

对分词结果进行解码：失去的东西就要学着去接受，学着放下。 (移除特殊字符)


## 参考资料

- [Hugging Face NLP Course](https://huggingface.co/learn/nlp-course/zh-CN/chapter2/4)

https://huggingface.co/Qwen/Qwen2.5-32B-Instruct-GPTQ-Int8/tree/main

Qwen/Qwen2.5-32B-Instruct-GPTQ-Int8

tokenizer_config.json
```json
"model_max_length": 131072,
chat_template="
{%- if tools %}
    {{- '<|im_start|>system\\n' }}
    {%- if messages[0]['role'] == 'system' %}
        {{- messages[0]['content'] }}
    {%- else %}
        {{- 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.' }}
    {%- endif %}
    {{- \"\\n\\n# Tools\\n\\nYou may call one or more functions to assist with the user query.\\n\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>\" }}
    {%- for tool in tools %}
        {{- \"\\n\" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- \"\\n</tools>\\n\\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\\n<tool_call>\\n{\\\"name\\\": <function-name>, \\\"arguments\\\": <args-json-object>}\\n</tool_call><|im_end|>\\n\" }}\n{%- else %}\n    
    {%- if messages[0]['role'] == 'system' %}
        {{- '<|im_start|>system\\n' + messages[0]['content'] + '<|im_end|>\\n' }}
    {%- else %}
        {{- '<|im_start|>system\\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\\n' }}
    {%- endif %}\n{%- endif %}\n{%- for message in messages %}
    {%- if (message.role == \"user\") or (message.role == \"system\" and not loop.first) or (message.role == \"assistant\" and not message.tool_calls) %}
        {{- '<|im_start|>' + message.role + '\\n' + message.content + '<|im_end|>' + '\\n' }}
    {%- elif message.role == \"assistant\" %}
        {{- '<|im_start|>' + message.role }}
    {%- if message.content %}
        {{- '\\n' + message.content }}
    {%- endif %}
    {%- for tool_call in message.tool_calls %}
        {%- if tool_call.function is defined %}
            {%- set tool_call = tool_call.function %}
        {%- endif %}
        {{- '\\n<tool_call>\\n{\"name\": \"' }}
        {{- tool_call.name }}
        {{- '\", \"arguments\": ' }}
        {{- tool_call.arguments | tojson }}
        {{- '}\\n</tool_call>' }}
    {%- endfor %}
    {{- '<|im_end|>\\n' }}
    {%- elif message.role == \"tool\" %}
        {%- if (loop.index0 == 0) or (messages[loop.index0 - 1].role != \"tool\") %}
            {{- '<|im_start|>user' }}
        {%- endif %}
        {{- '\\n<tool_response>\\n' }}
        {{- message.content }}
        {{- '\\n</tool_response>' }}   
        {%- if loop.last or (messages[loop.index0 + 1].role != \"tool\") %}        
            {{- '<|im_end|>\\n' }}    
        {%- endif %}
    {%- endif %}
{%- endfor %}
{%- if add_generation_prompt %}  
    {{- '<|im_start|>assistant\\n' }}
{%- endif %}
   ",
```
模板的结构分为几个部分。最外层检查是否有 tools 变量存在。如果有的话，生成系统消息，并包含工具的信息。如果没有，则根据 messages 里的第一个消息是否为系统消息来处理。接着遍历 messages 数组，根据不同的角色（user、assistant、tool）生成对应的部分。最后如果有生成提示的话，添加 assistant 的开头。




In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("../model/minimind_tokenizer")
print(tokenizer)
print(tokenizer.all_special_tokens)
print(tokenizer.special_tokens_map)

/home/hey/anaconda3/envs/minimind/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PreTrainedTokenizerFast(name_or_path='../model/minimind_tokenizer', vocab_size=6400, model_max_length=32768, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)
['<s>', '</s>', '<unk>']
{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}


In [2]:
msg = "失去的东西就要学着去接受，学着放下。"
tokens = tokenizer.tokenize(msg)
print(tokens)

['å¤±', 'åİ»', 'çļĦ', 'ä¸ľ', 'è¥¿', 'å°±', 'è¦ģ', 'åŃ¦', 'çĿĢ', 'åİ»', 'æİ¥åıĹ', 'ï¼Į', 'åŃ¦', 'çĿĢ', 'æĶ¾', 'ä¸ĭ', 'ãĢĤ']


In [36]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[2262, 1473, 269, 3600, 2540, 1343, 404, 595, 978, 1473, 4926, 270, 595, 978, 1192, 572, 286]


In [37]:
decoded_str = tokenizer.decode(ids)
print(decoded_str)

失去的东西就要学着去接受，学着放下。


In [41]:
tokenizer.chat_template

"{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{{ '<s>system\\n' + system_message + '</s>\\n' }}{% else %}{{ '<s>system\\n你是 MiniMind，是一个有用的人工智能助手。</s>\\n' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<s>user\\n' + content + '</s>\\n<s>assistant\\n' }}{% elif message['role'] == 'assistant' %}{{ content + '</s>' + '\\n' }}{% endif %}{% endfor %}"

model/minimind_tokenizer/tokenizer_config.json

chat_template="
    {% if messages[0]['role'] == 'system' %}
        {% set system_message = messages[0]['content'] %}
        {{ '<s>system\\n' + system_message + '</s>\\n' }}
    {% else %}
        {{ '<s>system\\n你是 MiniMind，是一个有用的人工智能助手。</s>\\n' }}
    {% endif %}
    {% for message in messages %}
        {% set content = message['content'] %}
        {% if message['role'] == 'user' %}
            {{ '<s>user\\n' + content + '</s>\\n<s>assistant\\n' }}
        {% elif message['role'] == 'assistant' %}
            {{ content + '</s>' + '\\n' }}
        {% endif %}
    {% endfor %}
"

<s>system
你是 MiniMind，是一个有用的人工智能助手。</s>
<s>user
content</s>
<s>assistant
content</s>

In [59]:
# 使用chat模板
msg = [{"role": "system", "content": "你是 xx，是一个有用的人工智能助手。"}]
new_msg = tokenizer.apply_chat_template(
    msg,
    tokenize=False
)
print(new_msg)

tokens = tokenizer.tokenize(new_msg)
ids = tokenizer.convert_tokens_to_ids(tokens)

# 移除特殊字符 # 比如在这个tokenizer中，<s>、</s>、<unk> 是特殊字符
decoded_str = tokenizer.decode(ids, skip_special_tokens=True)
print(decoded_str)

# 保留特殊字符
decoded_str = tokenizer.decode(ids, skip_special_tokens=False)
print(decoded_str)

<s>system
你是 xx，是一个有用的人工智能助手。</s>

system
你是 xx，是一个有用的人工智能助手。

<s>system
你是 xx，是一个有用的人工智能助手。</s>



In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("../model/minimind_tokenizer")
print(tokenizer)
print("all_special_tokens: ", tokenizer.all_special_tokens)
print("special_tokens_map: ", tokenizer.special_tokens_map)

msg = "失去的东西就要学着去接受，学着放下。"
tokens = tokenizer.tokenize(msg, add_special_tokens=True)
print(tokens)
tokens = tokenizer.tokenize(msg, add_special_tokens=False)
print(tokens)
# add_special_tokens=True 会添加特殊字符，add_special_tokens=False 不会添加特殊字符
# TODO 没效果，为何没加 <s> 和 </s> 呢？

print(tokenizer.tokenize("👍")) # BBPE 会把 emoji 拆分成字节，所以应该是不会有oov和<unk>的
print(tokenizer.tokenize("👍🏻"))


PreTrainedTokenizerFast(name_or_path='../model/minimind_tokenizer', vocab_size=6400, model_max_length=32768, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)
all_special_tokens:  ['<s>', '</s>', '<unk>']
special_tokens_map:  {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}
['å¤±', 'åİ»', 'çļĦ', 'ä¸ľ', 'è¥¿', 'å°±', 'è¦ģ', 'åŃ¦', 'çĿĢ', 'åİ»', 'æİ¥åıĹ', 'ï¼Į', 'åŃ¦', 'çĿĢ', 'æĶ¾', 'ä¸ĭ', 'ãĢĤ']
['å¤±', 'åİ»', 'çļĦ', 'ä¸ľ', 'è¥¿', 'å°±', 'è¦ģ', 'åŃ¦', 'çĿĢ', 'åİ»', 'æİ¥åıĹ',

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
# OSError: We couldn't connect to 'https://huggingface.co' to load this file
# 需要设置 HF_ENDPOINT=https://hf-mirror.com

print(tokenizer)
print("all_special_tokens: ", tokenizer.all_special_tokens)
print("special_tokens_map: ", tokenizer.special_tokens_map)

msg = "hello, world, 我是达文西!"
tokens = tokenizer.tokenize(msg, add_special_tokens=True)
print(tokens)
tokens = tokenizer.tokenize(msg, add_special_tokens=False)
print(tokens)
# add_special_tokens 有效果了，在tokenization时首尾添加了 [CLS] 和 [SEP]

BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)
all_special_tokens:  ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']
special_tokens_map:  {'unk_token': '[UNK]', 'sep_token': '[S